In [ ]:
import time
import sys
import os
from collections import OrderedDict
from importlib import reload

import h5py
import numpy as np
from matplotlib import pyplot as plt

%matplotlib notebook

In [ ]:
import qcodes
qc = qcodes

In [ ]:
from stationq.experiment import ATS; reload(ATS)
from stationq.experiment.ATS import AlazarMeasurement, AlzTimeTrace

In [ ]:
%run -n ../Experiments/init_vars.py
%run -n ../Experiments/init_defs.py

In [ ]:
from stationq.qctools import instruments as instools

from qcodes.instrument_drivers.AlazarTech.ATS9360 import AlazarTech_ATS9360
alazar = instools.create_inst(AlazarTech_ATS9360, 'alazar')

from qcodes.instrument_drivers.rigol.DG4000 import Rigol_DG4000
fg = instools.create_inst(Rigol_DG4000, 'fg', address="TCPIP0::169.254.190.44::inst0::INSTR")

from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = instools.create_inst(Tektronix_AWG5014, 'awg', address="TCPIP0::169.254.183.196::inst0::INSTR")

station = qc.Station(fg, alazar, awg)

In [ ]:
class AlazarDCMeasurement(AlazarMeasurementExt):
    
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)
        
        
class AWGMultiRamp(AWGMeasurement):
    
    trigger_chan = 'ch2_m1'
    ramp_chan = (2, 1)
    ramp_pts = (3, 3)
    ramp_min = (0, 0)
    ramp_max = (0.1, 0.1)
    ramp_elt_len = 10e-6
    
    def awg_sequence(self):
        h = self.awghandler
        
        chans = list(self.ramp_chan)
        if int(self.trigger_chan[2]) not in chans:
            chans.append(int(self.trigger_chan[2]))
        chans.sort()
    
        dt = h.get_awg_wf_dtype(chans=chans)
        nsamples = int(h.time_to_samples(self.ramp_elt_len))
        nreps = 1
        wfs = []
        
        idxs = np.meshgrid(*[np.arange(n) for n in self.ramp_pts], indexing='ij')
        vals = np.meshgrid(*[np.linspace(self.ramp_min[i], self.ramp_max[i], self.ramp_pts[i]) for i in range(len(self.ramp_chan))])
        
        for i in range(idxs[0].size):
            _vals = [v.reshape(-1)[i] for v in vals]
            _idxs = [idx.reshape(-1)[i] for idx in idxs]
            wf = np.zeros(nsamples, dtype=dt)
            for j, c in enumerate(self.ramp_chan):
                wf[f'ch{c}_wf'] = _vals[j]
            wf[self.trigger_chan][1:2] = 1
            
            wf_dict = dict(wf=wf, name='wf_{}'.format('-'.join([str(k) for k in _idxs])),
                           nreps=nreps)
            
            wfs.append(wf_dict)

        return wfs


class AlazarMultiRampTest(AlazarDCMeasurement, AWGMultiRamp):
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)
        

class AWG2DRamp(AWGMeasurement):
    
    trigger_chan = 'ch2_m1'
    ramp_chan = (2, 1)
    ramp_pts = (3, 3)
    ramp_min = (0, 0)
    ramp_max = (1, 0.1)
    ramp_elt_len = 10e-6
    ramp_down_len = 1e-3
    
    def awg_sequence(self):
        h = self.awghandler
        
        chans = list(self.ramp_chan)
        if int(self.trigger_chan[2]) not in chans:
            chans.append(int(self.trigger_chan[2]))
        chans.sort()
        
        dt = h.get_awg_wf_dtype(chans=chans)
        samples_per_val = int(h.time_to_samples(self.ramp_elt_len))
        samples_down = h.time_to_samples(self.ramp_down_len)
        samples_per_line = samples_per_val * self.ramp_pts[0] + samples_down
        
        wfs = []
        for i, v in enumerate(np.linspace(h.voltage_to_wfscale()[self.ramp_chan[1]-1] * self.ramp_min[1], 
                                          h.voltage_to_wfscale()[self.ramp_chan[1]-1] * self.ramp_max[1], 
                                          self.ramp_pts[1])):            
            wf = np.zeros(samples_per_line, dtype=dt)
            
            for j, w in enumerate(np.linspace(h.voltage_to_wfscale()[self.ramp_chan[0]-1] * self.ramp_min[0], 
                                              h.voltage_to_wfscale()[self.ramp_chan[0]-1] * self.ramp_max[0], 
                                              self.ramp_pts[0])):                
                wf[f'ch{self.ramp_chan[0]}_wf'][j*samples_per_val:(j+1)*samples_per_val] = w
                wf[f'{self.trigger_chan}'][j*samples_per_val+1:j*samples_per_val+2] = 1
            
            dn = np.linspace(h.voltage_to_wfscale()[self.ramp_chan[0]-1] * self.ramp_max[0], 
                             h.voltage_to_wfscale()[self.ramp_chan[0]-1] * self.ramp_min[0], 
                             samples_down)
            
            wf[f'ch{self.ramp_chan[0]}_wf'][-samples_down:] = dn
            wf[f'ch{self.ramp_chan[1]}_wf'][:] = v
            
            wfs.append(dict(wf=wf, name=f'wf_{i}'))
            
        # ramp outer value down as well
        wf = np.zeros(samples_down, dtype=dt)
        dn = np.linspace(h.voltage_to_wfscale()[self.ramp_chan[1]-1] * self.ramp_max[1], 
                         h.voltage_to_wfscale()[self.ramp_chan[1]-1] * self.ramp_min[1], 
                         samples_down)
        wf[f'ch{self.ramp_chan[1]}_wf'][:] = dn
        wfs.append(dict(wf=wf, name=f'wf_{i+1}'))
        
        return wfs
    

class Alazar2DRampTest(AlazarMeasurementExt, AWG2DRamp):
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)

In [ ]:
namespace.ats_settings['sample_rate'] = int(1e8)
namespace.ats_settings['trigger_source1'] = 'EXTERNAL'
namespace.awg_settings['sampling_rate'] = int(1e7)
namespace.awg_settings['channel_1']['analog_amplitude'] = 0.2
namespace.awg_settings['channel_2']['analog_amplitude'] = 0.2
fg.ch1_frequency(50)
fg.ch1_output_enabled(False)

m = Alazar2DRampTest(station, namespace)
m.trigger_src = fg
m.trigger_chan = 'ch2_m1'

m.ramp_chan = (2, 1)
m.ramp_pts = (101, 101)
m.ramp_min = (0, 0)
m.ramp_max = (0.1, 0.1)
m.ramp_elt_len = 85e-6
m.ramp_down_len = 100e-6

m.ats_int_time(80e-6)
m.ats_integrate_samples(True)
m.ats_average_records(False)
m.ats_average_buffers(False)
m.ats_records_per_buffer(m.ramp_pts[0])
m.ats_buffers_per_acquisition(m.ramp_pts[1])
m.ats_demod(True)
m.IF(10e6)

m.do_setup_awg = False
m.setup()

print(m.samples_per_record)

In [ ]:
data = m.acquire()

In [ ]:
A = data[0].reshape(101, 101)
B = data[1].reshape(101, 101)
A.dtype

In [ ]:
fig, ax = plt.subplots(1, 1)
im = ax.imshow(A.real, aspect='auto')
fig.colorbar(im)

fig, ax = plt.subplots(1, 1)
ax.imshow(B.real, aspect='auto')
fig.colorbar(im)

In [ ]:
2 * 2 * 82e2 * 101 * 101 * 1e-6